In [1]:
import numpy as np
import pandas as pd
import csv
import math
import seaborn as sns
import matplotlib.pyplot as plt
import numpy_financial as npf
import datetime

In [2]:
def homeAmount():
    print("Enter Home Amount Rounded to Nearest Whole Dollar (1 to 576000):")
    print("Example: If Home Price is $249,999.99, it should be input as 250000")            
    while True:
        try:
            #user input must be an integer
            number = int(input())
            #user input integer must be between 1 and 576000
            if 1 <= number <= 576000:
                return(number)
                break
            #if input is not an integer and/or it isn't between 1 and 576000, ask for user to input again
            raise ValueError()
        except ValueError:
            print("Enter Whole Number between 1 and 576000")
            
            
            
            

In [3]:
#downPayment asks user to input down payment, checks conditions, and returns integer
def downPayment():
    print("Enter Downpayment as a Percentage of Home Value Rounded to Nearest Whole Number (1 to 35):")
    print("Example: If Down Payment is 20%, it should be input as 20")
    while True:
        try:
            #user input must be an integer
            number = int(input())
            #user input integer must be between 1 and 35
            if 1 <= number <= 35:
                return(number)
                break
            raise ValueError()
        except ValueError:
            print("Enter Whole Number between 1 and 35")

In [4]:
##fix 2.44      number*10 is int
#interestRate asks user to input interest rate, checks conditions, and returns float
def interestRate():
    print("Enter Interest Rate as an Annual Percentage Rounded to the Nearest Tenth (1.0 to 3.5):")
    print("Example: If Interest Rate is 2%, it should be input as 2.0")
    while True:
        try:        
            number = float(input())
            y = str(number).split('.')
            if len(y[-1]) != 1:
                raise ValueError()
            #check = number*10
            #if check is int, good
            if 1.0 <= number <= 3.5:
                return(number)
                break
            raise ValueError()
        except ValueError:
            print("Enter Number with 1 Decimal between 1.0 and 3.5")
    

In [5]:
        

#loanLength asks user to input length of loan in years, checks conditions, and returns int
def loanLength():
    print("Enter Length of Loan in Years Rounded to Nearest Whole Year (5 - 30):")
    print("Example: If Loan Length is 20 years, it should be input as 20")
    while True:
        try:        
            number = int(input())
            if 5 <= number <= 30:
                return(number)
                break
            raise ValueError()
        except ValueError:
            print("Enter Whole Number between 5 and 30")

In [6]:
#numberPayments asks user to input the payment frequency per year, allows 1 of 3 choices, and returns int
def numberPayments():
    print("Enter Number of Payments per Year (12, 24, or 26):")
    print("Enter 12 for 1 Payment per Month, 24 for 2 Payments per Month, or 26 for 1 Payment Every 2 Weeks")
    while True:
        try:        
            number = int(input())
            if number == 12:
                return(number)
                break
            elif number == 24:
                return(number)
                break
            elif number == 26:
                return(number)
                break
            raise ValueError()
        except ValueError:
            print("Enter 12, 24, or 26")
            

In [7]:
            
#repaymentStartDate asks user to input year, month, and date as the start date of the loan, checks if date is valid, and returns datetime object
def repaymentStartDate():
    print("Enter Start Date of Loan Repayment in 3 Parts (Year,Month,Day):")
    while True:
        try:        
            year = int(input("Enter the Year (Example:2022) "))
            if 2020 <= year:
                month = int(input("Enter the Month (Example:February = 2) "))
                if 1 <= month <=12:
                    day=int(input("Enter the Day (Example 13th = 13) "))
                    #checks if date is valid for any given year, month, so that it rejects September 31 or February 30 etc
                    maxdate=datetime.datetime(year, (month+1)%12, 1) - datetime.timedelta(days=1)
                    if 1 <= day <= int(maxdate.day):
                        
                        return datetime.datetime(year,month,day)
                        break
                    raise ValueError()
                raise ValueError()
            raise ValueError()
        except ValueError:
            print("Invalid Year, Month, or Day")  

In [8]:
#extraPayment asks user for additional payments made on loan at their regularly scheduled payment interval, checks for conditions, returns int
def extraPayment():
    print("Enter Additional Payment per Pay Pay Period Rounded to Nearest Whole Dollar:")
    print("Example: If Extra Payment is $1250.00, it should be input as 1250")
    print("Enter 0 if you do not want to add any extra payments")
    while True:
        try:        
            number = int(input())
            if 0 <= number:
                return(number)
                break
            raise ValueError()
        except ValueError:
            print("Enter Whole Number 0 or greater")

In [9]:
#loanQualifier takes the down_payment and principal loan amount, and determines if the application qualifies for 0,1,2, or 3 loan types, returns string with answer
def loanQualifier(down_payment,principal):
    #create empty list and append elements to list if conditions are met
    list_qualifiers=[]
    if principal < 300000:
        if down_payment > 2:
            list_qualifiers.append("FHA")
    if principal < 480000:
        list_qualifiers.append("VA")
        if down_payment > 3.5:
            list_qualifiers.append("Conventional")
    #convert list to strings for human readability        
    st=' and '.join(list_qualifiers)
    st= 'This application qualifies for '+str(len(list_qualifiers))+' loans: '+st
    return st

In [10]:
#mortgageGenerator is a generator function used for calculations in mortgageCalculator function below:
def mortgageGenerator(principal, rate, payment, additional_payment):
    amount = payment + additional_payment
    while principal > 0:
        period_interest = round(principal*rate,2)
        principal_reduction = min(principal, amount - period_interest)
        principal = principal - principal_reduction
        yield principal_reduction, period_interest, principal
        

In [11]:
#mortgageCalculator takes all financial date gathered from user input, performs several financial calculations, and returns the full payoff schedule as a pandas df        
def mortgageCalculator(interest_rate, years, payments_year, principal, addl_principal=0, start_date=datetime.datetime.today()):
    #Calculate the fixed payment 
    period_rate = interest_rate/(payments_year * 100)
    payment = round(-1 * npf.pmt(period_rate, years*payments_year, principal),2)
    
    #Generate the period details schedule
    df = pd.DataFrame(list(mortgageGenerator(principal, period_rate, payment, addl_principal)), 		columns=["Principal", "Interest", "Ending Balance"])
    df["Extra Payment"] = addl_principal
    df["Total Payment"] = df["Principal"] + df["Interest"]
    
    #the freq argument changes depending on the number of payments per year payment
    if payments_year==12:
        payment_dates = pd.date_range(start_date, periods=len(df.index), freq='M')
    elif payments_year == 24:
        payment_dates = pd.date_range(start_date, periods=len(df.index), freq='SM')
    elif payments_year == 26:
        payment_dates = pd.date_range(start_date, periods=len(df.index), freq='2W')
        
    
    
    
    
    
    df.insert(0,"Payment Date", payment_dates)
    df.index += 1
    df.index.name = "Payment Number"
    df['Scheduled Payment']=payment
    df['Beginning Balance']=df['Ending Balance'].shift(1)
    df['Beginning Balance'].iloc[0]=principal
    if df['Beginning Balance'].loc[df.shape[0]]<df['Scheduled Payment'].loc[df.shape[0]]:
        df['Extra Payment'].loc[df.shape[0]]=0
        df['Total Payment'].loc[df.shape[0]]=df['Beginning Balance'].loc[df.shape[0]]
    else:
        df['Extra Payment'].loc[df.shape[0]]=df['Total Payment'].loc[df.shape[0]]-payment
    return(df)

         

In [12]:
#run all functions to gather user import for loan application
home_amount=homeAmount()            
down_payment=downPayment()
principal=home_amount-down_payment*home_amount/100            
interest_rate=interestRate()  
years=loanLength()      
payments_year=numberPayments()            
start_date=repaymentStartDate()             
addl_principal = extraPayment()

Enter Home Amount Rounded to Nearest Whole Dollar (1 to 576000):
Example: If Home Price is $249,999.99, it should be input as 250000
240000
Enter Downpayment as a Percentage of Home Value Rounded to Nearest Whole Number (1 to 35):
Example: If Down Payment is 20%, it should be input as 20
12
Enter Interest Rate as an Annual Percentage Rounded to the Nearest Tenth (1.0 to 3.5):
Example: If Interest Rate is 2%, it should be input as 2.0
2.0
Enter Length of Loan in Years Rounded to Nearest Whole Year (5 - 30):
Example: If Loan Length is 20 years, it should be input as 20
12
Enter Number of Payments per Year (12, 24, or 26):
Enter 12 for 1 Payment per Month, 24 for 2 Payments per Month, or 26 for 1 Payment Every 2 Weeks
12
Enter Start Date of Loan Repayment in 3 Parts (Year,Month,Day):
Enter the Year (Example:2022) 2022
Enter the Month (Example:February = 2) 2
Enter the Day (Example 13th = 13) 2
Enter Additional Payment per Pay Pay Period Rounded to Nearest Whole Dollar:
Example: If Extra P

In [14]:
#check which loan types the application qualifies for 
loanQualifier(down_payment,principal)

'This application qualifies for 3 loans: FHA and VA and Conventional'